# Computation with Xarray

- Aggregation: calculation of statistics (e.g. sum) along a dimension of an xarray object can be done by dimension name instead of an integer axis number.
- Arithmetic: arithmetic between xarray objects vectorizes based on dimension names, automatically looping (broadcasting) over each distinct dimension. This eliminates the need to insert dummy dimensions of size one to facilitate broadcasting, a common pattern with NumPy.
- Split-apply-combine: xarray includes N-dimensional grouped operations implementing the split-apply-combine strategy [24].
- Resampling and rolling window operations: Utilizing the efficient resampling methods from pandas and rolling window operations from Bottleneck [15], xarray offers a robust set of resampling and rolling window operations along a single dimension.


### Outline
- Arithmetic
- Aggregation
- Groupby and resample
- Rolling
- Universal Functions

### Tutorial Duriation
10 minutes

### Going Further

- Xarray documentation on its Computation Toolkit: http://xarray.pydata.org/en/latest/computation.html
- Xarray documentation on Groupby: http://xarray.pydata.org/en/latest/groupby.html

In [1]:
import numpy as np
import xarray as xr

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
ds = xr.tutorial.load_dataset('rasm')
da = ds['Tair']

ds

<xarray.Dataset>
Dimensions:  (time: 36, x: 275, y: 205)
Coordinates:
  * time     (time) datetime64[ns] 1980-09-16T12:00:00 1980-10-17 ...
    xc       (y, x) float64 189.2 189.4 189.6 189.7 189.9 190.1 190.2 190.4 ...
    yc       (y, x) float64 16.53 16.78 17.02 17.27 17.51 17.76 18.0 18.25 ...
Dimensions without coordinates: x, y
Data variables:
    Tair     (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
Attributes:
    title:                     /workspace/jhamman/processed/R1002RBRxaaa01a/l...
    institution:               U.W.
    source:                    RACM R1002RBRxaaa01a
    output_frequency:          daily
    output_mode:               averaged
    convention:                CF-1.4
    references:                Based on the initial model of Liang et al., 19...
    comment:                   Output from the Variable Infiltration Capacity...
    nco_openmp_thread_number:  1
    NCO:                       "4.6.0"
    history:                   Tue Dec 2

## Aggregation

Xarray supports many of the aggregations methods that numpy has. A partial list includes: all, any, argmax, argmin, max, mean, median, min, prod, sum, std, var.

Whereas the numpy syntax would require scalar axes, xarray can use dimension names:

In [3]:
da.mean(dim=('x', 'y'))

<xarray.DataArray 'Tair' (time: 36)>
array([  8.187292,  -0.701279,  -8.989001, -13.33163 , -17.543701, -13.953285,
        -9.637722,  -1.184811,   7.132873,  13.952738,  16.384343,  14.068569,
         7.338234,  -0.829985,  -9.569686, -15.55521 , -14.894763, -13.627037,
        -7.330606,  -0.204381,   7.476237,  14.240214,  16.800985,  14.197385,
         7.263972,  -1.248844,  -9.65014 , -14.568823, -18.896039, -14.423475,
        -9.778501,  -0.375954,   7.349978,  13.93472 ,  16.746357,  13.864375])
Coordinates:
  * time     (time) datetime64[ns] 1980-09-16T12:00:00 1980-10-17 ...

## Arithmetic

Arithmetic operations with a single DataArray automatically vectorize (like numpy) over all array values:

In [4]:
# dataarray + scalars
da - 273.15  # (K --> C)

<xarray.DataArray 'Tair' (time: 36, y: 205, x: 275)>
array([[[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ..., -246.347381, -246.063965],
        [        nan,         nan, ..., -246.585261, -246.419351]],

       [[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ..., -248.85376 , -248.535776],
        [        nan,         nan, ..., -248.850323, -248.695601]],

       ...,

       [[        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan],
        ...,
        [        nan,         nan, ..., -245.838951, -245.476128],
        [        nan,         nan, ..., -246.141106, -245.91982 ]],

       [[        nan,         nan, ...,         nan,         nan],
        [        n

Here we do two computations:
1. Calculate the time-mean
2. Calculate the "anomalies" relative to the time mean

In [5]:
da_mean = da.mean(dim='time')
da_mean

<xarray.DataArray 'Tair' (y: 205, x: 275)>
array([[      nan,       nan,       nan, ...,       nan,       nan,       nan],
       [      nan,       nan,       nan, ...,       nan,       nan,       nan],
       [      nan,       nan,       nan, ...,       nan,       nan,       nan],
       ...,
       [      nan,       nan,       nan, ..., 20.93361 , 20.807458, 21.025313],
       [      nan,       nan,       nan, ..., 21.170736, 20.566793, 20.836219],
       [      nan,       nan,       nan, ..., 20.862633, 20.515146, 20.688632]])
Coordinates:
    xc       (y, x) float64 189.2 189.4 189.6 189.7 189.9 190.1 190.2 190.4 ...
    yc       (y, x) float64 16.53 16.78 17.02 17.27 17.51 17.76 18.0 18.25 ...
Dimensions without coordinates: y, x

In [6]:
# dataarray + dataarray
da - da_mean  

# Notice that this required broadcasting along the time dimension

<xarray.DataArray 'Tair' (time: 36, y: 205, x: 275)>
array([[[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ..., 6.235826, 6.249816],
        [     nan,      nan, ..., 6.049592, 6.042018]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ..., 3.729447, 3.778006],
        [     nan,      nan, ..., 3.784531, 3.765768]],

       ...,

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ..., 6.744256, 6.837653],
        [     nan,      nan, ..., 6.493748, 6.541548]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [     nan,      nan, ..., 7.855943, 7.850993],
        [     nan,      nan, ..., 7.670809, 7.51889

## Groupby

xarray supports “group by” operations with the same API as pandas to implement the split-apply-combine strategy:

- Split your data into multiple independent groups.
- Apply some function to each group.
- Combine your groups back into a single data object.

Group by operations work on both Dataset and DataArray objects. Most of the examples focus on grouping by a single one-dimensional variable, although support for grouping over a multi-dimensional variable has recently been implemented.

In [7]:
# Using groupby to calculate a monthly climatology:

da_climatology = da.groupby('time.month').mean('time')

da_climatology

<xarray.DataArray 'Tair' (month: 12, y: 205, x: 275)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 10.055539, 10.309194],
        [      nan,       nan, ..., 10.413894, 10.620974]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 11.70764 , 11.874219],
        [      nan,       nan, ..., 11.730089, 11.881359]],

       ...,

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 15.585375, 15.865147],
        [      nan,       nan, ..., 15.868094, 16.082061]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 11.6483  , 11

In this case, we provide what we refer to as a virtual variable (`time.month`). Other virtual variables include:
“year”, “month”, “day”, “hour”, “minute”, “second”, “dayofyear”, “week”, “dayofweek”, “weekday” and “quarter”. It is also possible to use another DataArray or pandas object as the grouper.

## Rolling Operations

Xarray objects include a rolling method to support rolling window aggregations:

In [8]:
roller = da.rolling(time=3)
roller

DataArrayRolling [window->3,center->False,dim->time]

In [9]:
roller.mean()

<xarray.DataArray (time: 36, y: 205, x: 275)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan]],

       ...,

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 26.203183, 26.536968],
        [      nan,       nan, ..., 25.831169, 26.028754]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 27.925781, 28.235554]

In [10]:
# we can also provide a custom function 

def sum_minus_2(da, axis):
    return da.sum(axis=axis) - 2

roller.reduce(sum_minus_2)

<xarray.DataArray (time: 36, y: 205, x: 275)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan]],

       ...,

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 76.609548, 77.610903],
        [      nan,       nan, ..., 75.493507, 76.086263]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 81.777343, 82.706662]

## Universal Functions

In practice, not all use standard functions/methods from numpy and xarray is not allways possible. Sometimes there is good reason to work with numpy/dask array's directly. 

Xarray's documentation on wrapping custom computations: http://xarray.pydata.org/en/latest/computation.html#wrapping-custom-computation

In [11]:
da_noise = da + np.random.random(size=da.shape)
da_noise

<xarray.DataArray 'Tair' (time: 36, y: 205, x: 275)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 27.573181, 27.090213],
        [      nan,       nan, ..., 27.254112, 27.560609]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 24.522632, 25.450458],
        [      nan,       nan, ..., 24.771014, 25.194277]],

       ...,

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 27.536035, 28.258036],
        [      nan,       nan, ..., 27.95188 , 28.006402]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [      nan,       nan, ..., 29.002093, 29.

In [12]:
# some example legacy code to calculate the spearman correlation coefficient

import bottleneck


def covariance_gufunc(x, y):
    return ((x - x.mean(axis=-1, keepdims=True))
            * (y - y.mean(axis=-1, keepdims=True))).mean(axis=-1)

def correlation_gufunc(x, y):
    return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))

def spearman_correlation_gufunc(x, y):
    x_ranks = bottleneck.rankdata(x, axis=-1)
    y_ranks = bottleneck.rankdata(y, axis=-1)
    return correlation_gufunc(x_ranks, y_ranks)

In [14]:
# Wrap the "legacy code" with xarray's apply_ufunc. 
def spearman_correlation(x, y, dim):
    return xr.apply_ufunc(
        spearman_correlation_gufunc, x, y,
        input_core_dims=[[dim], [dim]],
        dask='parallelized',
        output_dtypes=[float])

In [15]:
da_corr = corr = spearman_correlation(da, da_noise, 'time')
da_corr

<xarray.DataArray 'Tair' (y: 205, x: 275)>
array([[1.      , 1.      , 1.      , ..., 1.      , 1.      , 1.      ],
       [1.      , 1.      , 1.      , ..., 1.      , 1.      , 1.      ],
       [1.      , 1.      , 1.      , ..., 1.      , 1.      , 1.      ],
       ...,
       [1.      , 1.      , 1.      , ..., 0.995624, 0.994337, 0.995367],
       [1.      , 1.      , 1.      , ..., 0.994595, 0.995882, 0.996654],
       [1.      , 1.      , 1.      , ..., 0.994337, 0.992021, 0.995109]])
Coordinates:
    xc       (y, x) float64 189.2 189.4 189.6 189.7 189.9 190.1 190.2 190.4 ...
    yc       (y, x) float64 16.53 16.78 17.02 17.27 17.51 17.76 18.0 18.25 ...
Dimensions without coordinates: y, x

## Masking

Indexing methods on xarray objects generally return a subset of the original data. However, it is sometimes useful to select an object with the same shape as the original data, but with some elements masked. To do this type of selection in xarray, use where():

In [22]:
# mask out 1's in the correlation array

da_corr.where(da_corr < 1)

<xarray.DataArray 'Tair' (y: 205, x: 275)>
array([[     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       [     nan,      nan,      nan, ...,      nan,      nan,      nan],
       ...,
       [     nan,      nan,      nan, ..., 0.995624, 0.994337, 0.995367],
       [     nan,      nan,      nan, ..., 0.994595, 0.995882, 0.996654],
       [     nan,      nan,      nan, ..., 0.994337, 0.992021, 0.995109]])
Coordinates:
    xc       (y, x) float64 189.2 189.4 189.6 189.7 189.9 190.1 190.2 190.4 ...
    yc       (y, x) float64 16.53 16.78 17.02 17.27 17.51 17.76 18.0 18.25 ...
Dimensions without coordinates: y, x

In [23]:
# xarray also provides a function for where
xr.where(da_corr > 0.996, 0, 1)

<xarray.DataArray 'Tair' (y: 205, x: 275)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 1]])
Coordinates:
    xc       (y, x) float64 189.2 189.4 189.6 189.7 189.9 190.1 190.2 190.4 ...
    yc       (y, x) float64 16.53 16.78 17.02 17.27 17.51 17.76 18.0 18.25 ...
Dimensions without coordinates: y, x